<a href="https://colab.research.google.com/github/ricotta-jpgomes/mod-multinivel-nuvem/blob/main/Azure/AZURE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Catálogo de preços para VMs na Azure e suas configurações
---

**Autor**: João Paulo Gomes Ricotta

**Ano**: 2025


Este notebook consolida uma base de dados com preços e configurações de diferentes instâncias de VMs disponíveis na Microsoft Azure, e faz parte do trabalho de conclusão de curso para o MBA em Data Science e Analytics da USP-ESALQ. Os dados do catálogo são públicos, e podem ser consultados pela api de preços de varejo da Azure. As configurações de cada máquina, por sua vez, são obtidas pela api de gerenciamento de conta, sendo necessário a criação de uma conta e um usuário impessoal (aplicativo) dentro do provedor.

## Configuração do ambiente 💻

Instalação de bibliotecas externas

In [ ]:
!pip install python-dotenv # Biblioteca para acessar chaves de api, users e tokens de acesso

Carregando as bibliotecas necessárias

In [ ]:
# Carregamento e configuração das variáveis de ambiente
import json
import os
import random
import time
from datetime import datetime, timedelta, timezone

# Manipulação de dados
import numpy as np
import pandas as pd

# Requisição à API e tratamento das respostas
import requests as rq

# Utilitários
import pytz  # definição de fuso horário
from dotenv import find_dotenv, load_dotenv, set_key

### Carregando as credenciais para geração do token de acesso

As informações coletadas para consolidação desta base de dados provêm de duas APIs distintas: a **Retail Prices API** e a **Management API**, ambas disponibilizadas pela Azure. A Retail Prices API possui um endpoint aberto e permite consultar informações de catálogo aberto do provedor, como os recursos disponíveis e seu custo. A Management API por sua vez está vinculada à uma conta da Azure, e só pode ser acessada mediante autenticação. O método uutilizado nesta coleta foi a autenticação via conta de serviço (aplicativo), gerando um token de acesso. Esse token é gerado a partir das seguintes informações:
<br>
<br>

* **tenant ID**: é o identificador único do diretório da sua organização no Azure Active Directory (Azure AD). Ele identifica a instância específica do Azure AD associada à sua organização ou ao ambiente em que os recursos estão sendo gerenciados. Cada inquilino (tenant) pode conter múltiplos usuários, aplicativos e grupos.
<br>

* **client ID**: é o identificador único do aplicativo registrado no Azure AD. Quando você registra um aplicativo no Azure AD, ele recebe um client ID, que é usado para identificar esse aplicativo ao realizar autenticação ou acessar recursos protegidos pela Azure AD. No gerenciamento de acessos da Azure o client é aentidade que pode receber acesso às diversas APIs disponíveis na plataforma.
<br>

* **client secret**: é uma chave de segurança gerada para um aplicativo registrado no Azure AD, funcionando como uma "senha" do aplicativo. Ele é usado junto com o client ID para autenticar o aplicativo e provar sua identidade ao Azure AD.
<br>

Todas essas informações foram armazenadas em um arquivo de ambiente (credentials.env), para não serem expostas em código. Elas são recuperadas por meio da biblioteca dotenv.


In [ ]:
# Carregando variáveis de ambiente
load_dotenv('/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/Azure/azure_credentials.env')

True

In [ ]:
# Informações do aplicativo
tenant_id = os.environ['TENANT_ID']
client_id = os.environ['CLIENT_ID']
client_secret = os.environ['CLIENT_SECRET']

In [ ]:
# Encontra o arquivo .env no caminho especificado
# Se não encontrar, ele pode criar um arquivo .env no diretório especificado
env_file = find_dotenv("/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código")
if not env_file:
    env_file = '/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/.env'  # Cria o arquivo .env no diretório atual se não existir

In [ ]:
# Obtendo a data de extração dos dados
diff = timedelta(hours = -3)
tzone = timezone(diff)
extraction_date = datetime.now(tz=tzone) # obtendo a data e hora da extração dos dados
extraction_date = extraction_date.astimezone(pytz.timezone("America/Sao_Paulo"))

set_key(env_file, 'EXTRACTION_DATE', f'{extraction_date.strftime("%d-%m-%Y")}')
print(extraction_date)

2025-07-06 16:39:11.418052-03:00


In [ ]:
def gerar_token_az(tenant_id, client_id, client_secret):
  # URL do endpoint de token
  url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'

  # Dados do payload (enviados na requisição)
  payload = {
      'grant_type': 'client_credentials',
      'client_id': client_id,
      'client_secret': client_secret,
      'scope': 'https://management.azure.com/.default'
  }

  # Requisição para obter o token
  response = rq.post(url, data=payload)
  response_data = response.json()

  # Capturando o token
  access_token = response_data.get('access_token')

  # Verificando se o token foi obtido
  if not access_token:
      raise Exception('Falha ao obter o token de acesso')

  print('Token de Acesso:', access_token)
  return access_token

## Coleta dos Dados 🎣

### Primeira etapa: catálogo de VMs da Azure

A primeira etapa consistiu na coleta das informações de catálogo públicas utilizando a api de preços de varejo da Azure (Retail Prices API). Esta etapa foi desenvolvida em dois processos distintos: Uma requisição para obter os preços de catálogo para máquinas virtuais on-demand e uma segunda requisição para obter os preços de catálogo para máquinas virtuais sob reserva (1 e 3 anos de compromisso). Essas informações refletem as principais estratégias de precificação dos provedores de nuvem. Para ambas as coletas foi utilizada uma estratégia de *retry with backoff*, para contornar eventual indisponibilidade na API por conta do grande volume de chamadas.

<br>

> A estratégia de **"retry with backoff"** (tentativa com recuo) é um mecanismo de controle de erros usado em programação, especialmente em comunicação de rede, para lidar com falhas temporárias ou intermitentes. Em vez de repetir uma operação falha imediatamente, a estratégia insere um tempo de espera antes de cada nova tentativa. A característica principal do "backoff" é que esse tempo de espera aumenta a cada falha consecutiva, geralmente de forma exponencial. Isso evita que o sistema cliente sobrecarregue um serviço que já pode estar em dificuldades (como uma API ou banco de dados), dando-lhe tempo para se recuperar e aumentando a probabilidade de sucesso nas tentativas subsequentes.


<br>

- **Custo on demand**: é o custo pago diretamente pelo uso dos recursos. Nesse modelo, o usuário paga apenas pelo que utiliza.

<br>

- **Custo sob reserva**: é o custo pago pelos recursos baseado em um compromisso de uso firmado com o provedor. Nesse modelo, o usuário se compromete a pagar pelos recursos reervados de forma contínua por períodos pré-estabelecidos, geralmente 1 ou 3 anos. Geralmente as reservas apresentam custo relativo menor quando comparados ao custo sob demanda, porém, o usuário paga independente de eatar utilizando ou não os recursos reservados, portanto esse modelo é recomendado para cargas de trabalho estáveis que apresentam uso contínuo dos recursos para um período longo de tempo.    

Vamos consultar VMs (Virtual Machines) do catálgo no regime de cobrança por reservas (Reservation). Uma requisição a essa api retorna os seguintes campos:

1. **currencyCode**: Código da moeda em que os preços estão listados, como "USD" ou "EUR".
2. **tierMinimumUnits**: A quantidade mínima de unidades na faixa de preços aplicável.
3. **retailPrice**: O preço de varejo para o serviço.
4. **unitPrice**: O preço por unidade do serviço.
5. **armRegionName**: Nome da região da Azure onde o serviço está disponível.
6. **location**: Localização do datacenter da Azure onde o serviço está hospedado.
7. **effectiveStartDate**: Data em que o preço se torna efetivo.
8. **meterId**: Identificador único do medidor, relacionado ao serviço consumido.
9. **meterName**: Nome do medidor de recursos ou serviço.
10. **productId**: Identificador do produto para o qual o preço é aplicável.
11. **productName**: Nome do produto/serviço da Azure.
12. **skuId**: Identificador do SKU (Stock Keeping Unit), representando uma versão específica do produto.
13. **skuName**: Nome do SKU.
14. **serviceName**: Nome do serviço da Azure (ex: Virtual Machines, App Service).
15. **serviceFamily**: Categoria do serviço (ex: Compute, Storage).
16. **priceType**: Tipo de preço (ex: Consumption, ReservedInstance, SavingsPlan).
17. **unitOfMeasure**: Unidade de medida para o recurso (ex: hora, GB, transação).
18. **reservationTerm**: Termo da reserva (1 ano, 3 anos) para instâncias reservadas.

In [ ]:
# Varredura do catálogo da Azure utilizando a estratégia Retry with Backoff
query_ondemand = f"serviceName eq 'Virtual Machines' and priceType eq 'Consumption'" # Consulta para restringir o escopo da requisição (VMs on-demand)
query_reservation = f"serviceName eq 'Virtual Machines' and priceType eq 'Reservation'" # Consulta para restringir o escopo da requisição (VMs reservadas)

max_retries = 5  # Número máximo de tentativas por página
initial_backoff_seconds = 5  # Tempo de espera inicial após um erro 429

#### Requisição para preços on-demand

In [ ]:
api_url = "https://prices.azure.com/api/retail/prices?api-version=2021-10-01-preview" # url para requisição de dados à api
table_data = []
next_page = None

In [ ]:
# Primeira requisição
page = 1
try:
    response = rq.get(api_url, params={'$filter': query_ondemand})
    response.raise_for_status() # Lança um erro para status HTTP 4xx/5xx

    json_data = response.json()
    table_data.extend(json_data['Items'])
    nextPage = json_data.get('NextPageLink')

    print(f"Página {page} carregada com sucesso.")

except rq.exceptions.RequestException as e:
    print(f"Erro na primeira requisição, abortando. Erro: {e}")
    nextPage = None

Página 1 carregada com sucesso.


In [ ]:
# Laço principal para as páginas seguintes
while(nextPage):
    page += 1
    current_wait_time = initial_backoff_seconds
    # Laço de tentativas (Retry Loop) para a página atual
    for attempt in range(max_retries):

        try:
            print(f"Carregando página {page} (tentativa {attempt + 1}/{max_retries})...")
            response = rq.get(nextPage)

            # Se a resposta for 429 (Too Many Requests), vamos tratar como um erro para acionar o backoff
            if response.status_code == 429:
                # Criamos um erro artificial para ser pego pelo bloco 'except'
                raise rq.exceptions.HTTPError(f"Erro 429: Too Many Requests. Acionando backoff.")

            response.raise_for_status() # Lança erro para outros status 4xx/5xx

            # --- Sucesso ---
            json_data = response.json()
            table_data.extend(json_data['Items'])
            nextPage = json_data.get('NextPageLink') # Pega o link da próxima página

            print(f"Página {page} carregada com sucesso.")
            time.sleep(random.randint(1, 3)) # Pausa curta de 1 segundo em caso de sucesso
            break # Sai do laço de tentativas (for) e vai para a próxima página (while)

        except rq.exceptions.HTTPError as e:
            # --- Falha com Backoff ---
            wait_time = random.randint(current_wait_time, initial_backoff_seconds * (2 ** attempt)) # wait_time sempre será um valor aleatório entre o tempo de espera inicial e um Backoff Exponencial
            print(f"--> Erro HTTP: {e}")
            print(f"--> Aguardando {wait_time} segundos antes de tentar novamente...")
            current_wait_time = wait_time # Atualiza o tempo de espera atual para o próximo loop
            time.sleep(wait_time)

        except rq.exceptions.RequestException as e:
            # --- Falha de Conexão/Outros ---
            wait_time = random.randint(current_wait_time, initial_backoff_seconds * (2 ** attempt))
            print(f"--> Erro de Conexão/Requisição: {e}")
            print(f"--> Aguardando {wait_time} segundos antes de tentar novamente...")
            current_wait_time = wait_time
            time.sleep(wait_time)

    else: # Este 'else' pertence ao 'for'. É executado se o laço de tentativas terminar sem um 'break'
        print(f"Não foi possível carregar a página {page} após {max_retries} tentativas. Abortando a coleta.")
        print(f"O link para recomeçar a coleta é: {nextPage}")
        break # Sai do laço principal (while)

print("\nColeta de dados finalizada.")

Carregando página 2 (tentativa 1/5)...
Página 2 carregada com sucesso.
Carregando página 3 (tentativa 1/5)...
Página 3 carregada com sucesso.
Carregando página 4 (tentativa 1/5)...
Página 4 carregada com sucesso.
Carregando página 5 (tentativa 1/5)...
Página 5 carregada com sucesso.
Carregando página 6 (tentativa 1/5)...
Página 6 carregada com sucesso.
Carregando página 7 (tentativa 1/5)...
Página 7 carregada com sucesso.
Carregando página 8 (tentativa 1/5)...
Página 8 carregada com sucesso.
Carregando página 9 (tentativa 1/5)...
Página 9 carregada com sucesso.
Carregando página 10 (tentativa 1/5)...
Página 10 carregada com sucesso.
Carregando página 11 (tentativa 1/5)...
Página 11 carregada com sucesso.
Carregando página 12 (tentativa 1/5)...
Página 12 carregada com sucesso.
Carregando página 13 (tentativa 1/5)...
Página 13 carregada com sucesso.
Carregando página 14 (tentativa 1/5)...
Página 14 carregada com sucesso.
Carregando página 15 (tentativa 1/5)...
Página 15 carregada com suc

In [ ]:
df_vms_ondemand = pd.DataFrame(table_data)
print(f"Total de registros coletados: {len(df_vms_ondemand)}")

Total de registros coletados: 270738


In [ ]:
# Máquinas spot não serão o objeto da análise aqui, portanto iremos eliminar os registros referentes a esse tipo de máquina
df_vms_ondemand = df_vms_ondemand[~
                      (df_vms_ondemand['skuName'].str.contains('Spot')) |
                      (df_vms_ondemand['meterName']).str.contains('Low Priority')
]
print(f"Total de registros após filtro: {len(df_vms_ondemand)}")

Total de registros após filtro: 178854


In [ ]:
# Renomeando a coluna de custo
df_vms_ondemand.rename(columns={'retailPrice': 'priceOnDemand'}, inplace=True)

/tmp/ipython-input-14-291623853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vms_ondemand.rename(columns={'retailPrice': 'priceOnDemand'}, inplace=True)


#### Requisição para preços sob reserva

In [ ]:
api_url = "https://prices.azure.com/api/retail/prices?api-version=2021-10-01-preview" # url para requisição de dados à api
table_data = []
next_page = None

In [ ]:
# Primeira requisição
page = 1
try:
    response = rq.get(api_url, params={'$filter': query_reservation})
    response.raise_for_status() # Lança um erro para status HTTP 4xx/5xx

    json_data = response.json()
    table_data.extend(json_data['Items'])
    nextPage = json_data.get('NextPageLink')

    print(f"Página {page} carregada com sucesso.")

except rq.exceptions.RequestException as e:
    print(f"Erro na primeira requisição, abortando. Erro: {e}")
    nextPage = None

Página 1 carregada com sucesso.


In [ ]:
# Laço principal para as páginas seguintes
while(nextPage):
    page += 1
    current_wait_time = initial_backoff_seconds
    # Laço de tentativas (Retry Loop) para a página atual
    for attempt in range(max_retries):

        try:
            print(f"Carregando página {page} (tentativa {attempt + 1}/{max_retries})...")
            response = rq.get(nextPage)

            # Se a resposta for 429 (Too Many Requests), vamos tratar como um erro para acionar o backoff
            if response.status_code == 429:
                # Criamos um erro artificial para ser pego pelo bloco 'except'
                raise rq.exceptions.HTTPError(f"Erro 429: Too Many Requests. Acionando backoff.")

            response.raise_for_status() # Lança erro para outros status 4xx/5xx

            # --- Sucesso ---
            json_data = response.json()
            table_data.extend(json_data['Items'])
            nextPage = json_data.get('NextPageLink') # Pega o link da próxima página

            print(f"Página {page} carregada com sucesso.")
            time.sleep(random.randint(1, 3)) # Pausa curta de 1 segundo em caso de sucesso
            break # Sai do laço de tentativas (for) e vai para a próxima página (while)

        except rq.exceptions.HTTPError as e:
            # --- Falha com Backoff ---
            wait_time = random.randint(current_wait_time, initial_backoff_seconds * (2 ** attempt)) # wait_time sempre será um valor aleatório entre o tempo de espera inicial e um Backoff Exponencial
            print(f"--> Erro HTTP: {e}")
            print(f"--> Aguardando {wait_time} segundos antes de tentar novamente...")
            current_wait_time = wait_time # Atualiza o tempo de espera atual para o próximo loop
            time.sleep(wait_time)

        except rq.exceptions.RequestException as e:
            # --- Falha de Conexão/Outros ---
            wait_time = random.randint(current_wait_time, initial_backoff_seconds * (2 ** attempt))
            print(f"--> Erro de Conexão/Requisição: {e}")
            print(f"--> Aguardando {wait_time} segundos antes de tentar novamente...")
            current_wait_time = wait_time
            time.sleep(wait_time)

    else: # Este 'else' pertence ao 'for'. É executado se o laço de tentativas terminar sem um 'break'
        print(f"Não foi possível carregar a página {page} após {max_retries} tentativas. Abortando a coleta.")
        print(f"O link para recomeçar a coleta é: {nextPage}")
        break # Sai do laço principal (while)

print("\nColeta de dados finalizada.")

Carregando página 2 (tentativa 1/5)...
Página 2 carregada com sucesso.
Carregando página 3 (tentativa 1/5)...
Página 3 carregada com sucesso.
Carregando página 4 (tentativa 1/5)...
Página 4 carregada com sucesso.
Carregando página 5 (tentativa 1/5)...
Página 5 carregada com sucesso.
Carregando página 6 (tentativa 1/5)...
Página 6 carregada com sucesso.
Carregando página 7 (tentativa 1/5)...
Página 7 carregada com sucesso.
Carregando página 8 (tentativa 1/5)...
Página 8 carregada com sucesso.
Carregando página 9 (tentativa 1/5)...
Página 9 carregada com sucesso.
Carregando página 10 (tentativa 1/5)...
Página 10 carregada com sucesso.
Carregando página 11 (tentativa 1/5)...
Página 11 carregada com sucesso.
Carregando página 12 (tentativa 1/5)...
Página 12 carregada com sucesso.
Carregando página 13 (tentativa 1/5)...
Página 13 carregada com sucesso.
Carregando página 14 (tentativa 1/5)...
Página 14 carregada com sucesso.
Carregando página 15 (tentativa 1/5)...
Página 15 carregada com suc

In [ ]:
df_vms_reserved = pd.DataFrame(table_data)
print(f"Total de registros coletados: {len(df_vms_reserved)}")

Total de registros coletados: 92011


In [ ]:
df_vms_reserved.info()
display(df_vms_reserved.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92011 entries, 0 to 92010
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   currencyCode          92011 non-null  object 
 1   tierMinimumUnits      92011 non-null  float64
 2   reservationTerm       92011 non-null  object 
 3   retailPrice           92011 non-null  float64
 4   unitPrice             92011 non-null  float64
 5   armRegionName         92011 non-null  object 
 6   location              92011 non-null  object 
 7   effectiveStartDate    92011 non-null  object 
 8   meterId               92011 non-null  object 
 9   meterName             92011 non-null  object 
 10  productId             92011 non-null  object 
 11  skuId                 92011 non-null  object 
 12  productName           92011 non-null  object 
 13  skuName               92011 non-null  object 
 14  serviceName           92011 non-null  object 
 15  serviceId          

,currencyCode,tierMinimumUnits,reservationTerm,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,meterId,meterName,...,skuId,productName,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName
0,USD,0.0,1 Year,33229.0,33229.0,chilecentral,CL Central,2025-04-01T00:00:00Z,00010667-6f34-57aa-bbb3-3232a8e1b190,E64-16ds v4,...,DZH318Z0CSHK/0HS0,Virtual Machines Edsv4 Series,E64-16ds v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E64-16ds_v4
1,USD,0.0,3 Years,63746.0,63746.0,chilecentral,CL Central,2025-04-01T00:00:00Z,00010667-6f34-57aa-bbb3-3232a8e1b190,E64-16ds v4,...,DZH318Z0CSHK/0HRW,Virtual Machines Edsv4 Series,E64-16ds v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E64-16ds_v4
2,USD,0.0,1 Year,1406.0,1406.0,italynorth,IT North,2025-07-01T00:00:00Z,00018fd3-73fa-587a-a24f-c379036cede6,E4pds v6,...,DZH318Z0G1GS/0336,Virtual Machines Epdsv6 Series,E4pds v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E4pds_v6
3,USD,0.0,3 Years,2716.0,2716.0,italynorth,IT North,2025-07-01T00:00:00Z,00018fd3-73fa-587a-a24f-c379036cede6,E4pds v6,...,DZH318Z0G1GS/0337,Virtual Machines Epdsv6 Series,E4pds v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_E4pds_v6
4,USD,0.0,1 Year,58584.0,58584.0,canadaeast,CA East,2025-04-01T00:00:00Z,00029f02-43d4-5aa9-b882-cf4b159a2834,D192s v6,...,DZH318Z0GNHB/042H,Virtual Machines Dsv6 Series,D192s v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Reservation,True,Standard_D192s_v6


#### Estruturando o dataframe de preços

Para unificar as informações de preço em um único conjunto de dados, foi necessário proceder algumas transformações sobre os dados de preço para reservas, pivotando a tabela de modo que as informações sobre o período de compromisso fossem exibidas como colunas.  

In [ ]:
# Garante que a coluna de preço é numérica
df_vms_reserved['retailPrice'] = pd.to_numeric(df_vms_reserved['retailPrice'], errors='coerce')

# Define o identificador único de uma VM (nome + região)
vm_id = ['armSkuName', 'armRegionName', 'location', 'meterId']

# Pivota a tabela
df_vms_reserved = df_vms_reserved.pivot_table(
    index=vm_id,
    columns='reservationTerm',
    values='retailPrice'
).reset_index()

# Renomeia as colunas para um padrão claro e consistente
# Ajuste os nomes ('1 Year', '3 Years') para corresponder exatamente ao que a API retorna
df_vms_reserved.rename(columns={
    '1 Year': 'priceReserved1yr',
    '3 Years': 'priceReserved3yr'
}, inplace=True)

# Remove possíveis nomes de colunas indesejados após o pivot
df_vms_reserved.columns.name = None

print("Tabela de reservas pivotada com sucesso!")
df_vms_reserved.info()
display(df_vms_reserved.head())

Reestruturando a tabela de preços de reserva...
Tabela de reservas pivotada com sucesso!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45998 entries, 0 to 45997
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   armSkuName        45998 non-null  object 
 1   armRegionName     45998 non-null  object 
 2   location          45998 non-null  object 
 3   meterId           45998 non-null  object 
 4   priceReserved1yr  45998 non-null  float64
 5   priceReserved3yr  45833 non-null  float64
 6   5 Years           180 non-null    float64
dtypes: float64(3), object(4)
memory usage: 2.5+ MB


,armSkuName,armRegionName,location,meterId,priceReserved1yr,priceReserved3yr,5 Years
0,DCadsv5 Type 1,centralus,US Central,dd687ee2-13da-5e69-a9f3-c42e83ddd4a2,44464.0,100045.0,NaN
1,DCadsv5 Type 1,eastus,US East,d3729008-12b9-5235-ab15-31deab54a513,44464.0,100045.0,NaN
2,DCadsv5 Type 1,northeurope,EU North,732299bc-52e4-5dea-878b-79b5056ab186,49645.0,111701.0,NaN
3,DCadsv5 Type 1,westeurope,EU West,5ef44df0-e5ce-5516-97da-f0224be0f33e,53962.0,121414.0,NaN
4,DCadsv5 Type 1,westus,US West,e6bad145-e4ef-5eca-874b-69a3bbd27726,52667.0,118500.0,NaN


In [ ]:
df_vms_reserved.drop(columns=['5 Years'], inplace=True) # Algumas VMs possuem disponibilidade de compromisso para 5 anos, e naõ serão consideradas no contexto da pesquisa

In [ ]:
# Une os dois DataFrames usando o identificador da VM como chave
df_vms_prices = pd.merge(
    df_vms_ondemand,
    df_vms_reserved,
    on=['armSkuName', 'armRegionName', 'location', 'meterId'],
    how='left' #
)

print("Junção finalizada!")

# Exibe o resultado final com todas as colunas de preço
display(df_vms_prices[[
    'armSkuName',
    'armRegionName',
    'priceOnDemand',
    'priceReserved1yr',
    'priceReserved3yr'
]].head())

Unindo os preços de reserva ao DataFrame principal de VMs On-Demand...
Junção finalizada!


,armSkuName,armRegionName,priceOnDemand,priceReserved1yr,priceReserved3yr
0,Standard_E4-2as_v6,uaenorth,0.478,NaN,NaN
1,Standard_D16as_v4,switzerlandnorth,0.734,NaN,NaN
2,Standard_D16ds_v4,israelnorthwest,1.792,NaN,NaN
3,Standard_E64-16s_v5,francecentral,0.947,NaN,NaN
4,Standard_E64-16ds_v4,chilecentral,6.451,33229.0,63746.0


      
      

**priceReserved1yr** e **priceReserved3yr** representam o custo final da instância pelo termo de compromisso (reservationTerm) correspondente, que pode ser de 1 ou 3 anos (existem alguns registros com termo de compromisso de cinco anos mas são poucos casos e extrapolam o contexto da pesquisa).

**effectiveStartDate** representa o momento a partir do qual o valor da instância passou a valer, trazendo diversas datas que variam de 2017 a 2025. Elas não refletem a evolução histórica dos preços ano a ano, apenas uma data de referência para o valor cobrado por cada instância de VM. Cada ano possui um conjunto de instâncias específico que não se repete nos demais.

As informações obtidas nessa requisição foram validadas por meio da calculadora de preços da Azure, disponível em https://azure.microsoft.com/pt-br/pricing/calculator/

### Segunda etapa: detalhes de configuração para cada máquina

A versão do endpoint da Management API utilizada nesta etapa da coleta (2023-07-01), não oferece suporte a todas as regiões. Como os registros do dataframe de preços (df_vms_prices) servirão de filtro para as requisições à essa API, vamos filtrar apenas os registros em que a região se encontra entre as regiões suportadas, para não fazer chamadas desnecessárias à API.

In [ ]:
supported_locations = ['eastus', 'eastus2', 'westus', 'centralus', 'northcentralus', 'southcentralus', 'northeurope',
 'westeurope', 'eastasia', 'southeastasia', 'japaneast', 'japanwest', 'australiaeast',
 'australiasoutheast', 'australiacentral', 'brazilsouth', 'southindia', 'centralindia', 'westindia',
 'canadacentral', 'canadaeast', 'westus2', 'westcentralus', 'uksouth', 'ukwest', 'koreacentral',
 'koreasouth', 'francecentral', 'southafricanorth', 'uaenorth', 'switzerlandnorth', 'germanywestcentral',
 'norwayeast', 'jioindiawest', 'westus3', 'swedencentral', 'qatarcentral', 'polandcentral', 'italynorth',
 'israelcentral', 'spaincentral', 'mexicocentral']

In [ ]:
# Quantidade de regiões antes do filtro
regions = df_vms_prices.armRegionName.unique()

In [ ]:
df_vms_az = df_vms_prices[df_vms_az['armRegionName'].isin(supported_locations)]
print(f"Total de registros após filtro: {len(df_vms_az)}") # Registros após o filtro
print(f"\nTotal de regiões após o filtro: {len(df_vms_az.armRegionName.unique())}") # Regiões após o filtro

Total de registros após filtro: 134020

Total de regiões após o filtro: 42


#### Requisitando detalhes de configuração das VMs no catálogo

In [ ]:
token_az = gerar_token_az(tenant_id, client_id, client_secret) # Gerando o token de acesso

Token de Acesso: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Il9qTndqZVNudlRUSzhYRWRyNVFVUGtCUkxMbyIsImtpZCI6Il9qTndqZVNudlRUSzhYRWRyNVFVUGtCUkxMbyJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvYjkyYzhhOTItNzhkZS00ZGZmLWI5ZjctNWYzYWMwMTE4YTI3LyIsImlhdCI6MTc1MTgzMTU0NiwibmJmIjoxNzUxODMxNTQ2LCJleHAiOjE3NTE4MzU0NDYsImFpbyI6ImsyUmdZSkQ2a2hFNXI2VlVzSFNoVE5DUFhkZWxBQT09IiwiYXBwaWQiOiIxOWZiY2Q4NS0wZDUzLTRlMmMtOGYwNS05NzY5ZjRjODNlYzMiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9iOTJjOGE5Mi03OGRlLTRkZmYtYjlmNy01ZjNhYzAxMThhMjcvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiIwZTQxYmY0OC0zNTllLTRkYWUtOGQ0OC05NWM5ZGM4YjA2NWUiLCJyaCI6IjEuQVdFQmtvb3N1ZDU0XzAyNTkxODZ3QkdLSjBaSWYza0F1dGRQdWtQYXdmajJNQlBLQVFCaEFRLiIsInN1YiI6IjBlNDFiZjQ4LTM1OWUtNGRhZS04ZDQ4LTk1YzlkYzhiMDY1ZSIsInRpZCI6ImI5MmM4YTkyLTc4ZGUtNGRmZi1iOWY3LTVmM2FjMDExOGEyNyIsInV0aSI6Ii1IOEhZZENYY0VxUGZFOGNiWmdBQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfZnRkIjoiSGsxNEticVZ4bjJka0JIX2xkdFJOVlZFbEVKOFBkeFhSMHFRUE

In [ ]:
headers = {
    'Authorization': f'Bearer {token_az}',
    'Content-Type': 'application/json'
}

In [ ]:
table_data = []

for region in regions:
  api_url = f'https://management.azure.com/subscriptions/8f25fb89-b36b-4445-a7bd-7360d8453cab/providers/Microsoft.Compute/locations/{region}/vmSizes?api-version=2023-07-01'

  # Requisição GET para a API
  api_response = rq.get(api_url, headers=headers)
  #print(api_response.json())

  for config in api_response.json()['value']:
    table_data.append([config['name'], region, config['numberOfCores'], config['osDiskSizeInMB'], config['resourceDiskSizeInMB'], config['memoryInMB'], config['maxDataDiskCount']])

  wait_time = random.randint(1, 3)
  time.sleep(wait_time)

print("-> Coleta de dados finalizada.")
print(f"\n-> Detalhes de configuração recuperados para {len(table_data)} VMs.")

-> Coleta de dados finalizada.

-> Detalhes de configuração recuperados para 32867 VMs.


In [ ]:
df_vms_config = pd.DataFrame(table_data, columns=['name', 'region', 'numberOfCores', 'osDiskSizeInMB', 'resourceDiskSizeInMB', 'memoryInMB', 'maxDataDiskCount'])
df_vms_config.info()
display(df_vms_config.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32867 entries, 0 to 32866
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  32867 non-null  object
 1   region                32867 non-null  object
 2   numberOfCores         32867 non-null  int64 
 3   osDiskSizeInMB        32867 non-null  int64 
 4   resourceDiskSizeInMB  32867 non-null  int64 
 5   memoryInMB            32867 non-null  int64 
 6   maxDataDiskCount      32867 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 1.8+ MB


,name,region,numberOfCores,osDiskSizeInMB,resourceDiskSizeInMB,memoryInMB,maxDataDiskCount
0,Standard_B1ls,uaenorth,1,1047552,4096,512,2
1,Standard_B1ms,uaenorth,1,1047552,4096,2048,2
2,Standard_B1s,uaenorth,1,1047552,4096,1024,2
3,Standard_B2ms,uaenorth,2,1047552,16384,8192,4
4,Standard_B2s,uaenorth,2,1047552,8192,4096,4


1. **name**: Nome da instância. Equivalente a **armSkuName**

1. **region**: Região geográfica onde a máquina virtual (VM) está hospedada. É equivalente a **armRegionName**

2. **numberOfCores**: Número de núcleos de CPU disponíveis na máquina virtual. Indica a capacidade de processamento da VM (um número maior de núcleos permite maior capacidade de executar tarefas paralelas ou trabalhos computacionalmente intensivos)

3. **osDiskSizeInMB**: Especifica o tamanho do disco do sistema operacional em megabytes (MB). Esse é o disco principal onde o sistema operacional da VM está instalado e contém os arquivos essenciais para a inicialização e execução do sistema.

4. **resourceDiskSizeInMB**: Refere-se ao tamanho do disco de recursos temporários em megabytes (MB). Esse disco geralmente é usado para armazenamento temporário de dados e não deve ser utilizado para armazenamento persistente, pois seu conteúdo pode ser perdido durante eventos como reinicialização da VM.

5. **memoryInMB**: quantidade de memória RAM (em MB) alocada para a VM. Quanto maior a memória disponível, mais dados podem ser mantidos em cache e processados em paralelo, melhorando o desempenho em aplicações que demandam muitos recursos.

6. **maxDataDiskCount**: número máximo de discos de dados que podem ser anexados à VM. Esses discos são adicionais ao disco do sistema operacional e ao disco temporário, sendo utilizados para armazenar dados de maneira persistente.

#### Merge final (preços x configurações)

A última etapa do tratamento consiste em consolidar uma base única em que conste o catálogo com os preços das instâncias de VMs e os dados de configuração de cada uma. Isto será obtido a partir do merge entre os dois dataframes obtidos nas etapas anteriores. A Primary Key para os dois datasets correspondem ao nome da instância de VM + região, considerando que não existem duas instâncias iguais por região.

*   **PK em df_instances** = armSkuName + armRegionName
*   **PK em df_configs** = name + region

In [ ]:
# O método para realizar o merge será o inner, pois a base final não deve possuir nenhum campo nulo.
# Desta forma, devem ser considerados apenas registros comuns aos dois dataframes.
df_vms_az = pd.merge(df_vms_az, df_vms_config, left_on=['armSkuName', 'armRegionName'], right_on=['name', 'region'], how='inner')
df_vms_az.info()
display(df_vms_az.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109153 entries, 0 to 109152
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   currencyCode          109153 non-null  object 
 1   tierMinimumUnits      109153 non-null  float64
 2   priceOnDemand         109153 non-null  float64
 3   unitPrice             109153 non-null  float64
 4   armRegionName         109153 non-null  object 
 5   location              109153 non-null  object 
 6   effectiveStartDate    109153 non-null  object 
 7   meterId               109153 non-null  object 
 8   meterName             109153 non-null  object 
 9   productId             109153 non-null  object 
 10  skuId                 109153 non-null  object 
 11  productName           109153 non-null  object 
 12  skuName               109153 non-null  object 
 13  serviceName           109153 non-null  object 
 14  serviceId             109153 non-null  object 
 15  

,currencyCode,tierMinimumUnits,priceOnDemand,unitPrice,armRegionName,location,effectiveStartDate,meterId,meterName,productId,...,armSkuName,priceReserved1yr,priceReserved3yr,name,region,numberOfCores,osDiskSizeInMB,resourceDiskSizeInMB,memoryInMB,maxDataDiskCount
0,USD,0.0,0.7340,0.7340,switzerlandnorth,CH North,2022-12-01T00:00:00Z,7d3befd6-d304-5c79-aa4e-4b5450973b73,D16as v4 Low Priority,DZH318Z0CM93,...,Standard_D16as_v4,NaN,NaN,Standard_D16as_v4,switzerlandnorth,16,1047552,131072,65536,32
1,USD,0.0,0.9470,0.9470,francecentral,FR Central,2021-11-01T00:00:00Z,1d42e5ce-9bcb-52cd-bb52-3ad9a9863692,E64-16s v5 Low Priority,DZH318Z096SR,...,Standard_E64-16s_v5,NaN,NaN,Standard_E64-16s_v5,francecentral,64,1047552,0,524288,32
2,USD,0.0,4.2430,4.2430,mexicocentral,MX Central,2024-09-01T00:00:00Z,0001492f-d0d9-53e2-ba36-7b4b66b2f521,D96als v6,DZH318Z0F5F4,...,Standard_D96als_v6,NaN,NaN,Standard_D96als_v6,mexicocentral,96,1047552,0,196608,32
3,USD,0.0,0.2720,0.2720,italynorth,IT North,2025-07-01T00:00:00Z,00018fd3-73fa-587a-a24f-c379036cede6,E4pds v6,DZH318Z0G1GS,...,Standard_E4pds_v6,1406.0,2716.0,Standard_E4pds_v6,italynorth,4,1047552,0,32768,12
4,USD,0.0,0.0236,0.0236,westindia,IN West,2021-02-01T00:00:00Z,000419bc-6006-53ea-8976-1644249b9728,D2 v4 Low Priority,DZH318Z0D1L4,...,Standard_D2_v4,NaN,NaN,Standard_D2_v4,westindia,2,1047552,0,8192,4


## Tratamento 💆

In [ ]:
# Filtros
az_general_purpose_machines = ['Standard_A', 'Standard_B', 'Standard_D', 'Basic_A']
general_purpose_machines_expression = '|'.join(az_general_purpose_machines)

df_vms_az = df_vms_az.loc[
    (df_vms_az['isPrimaryMeterRegion'] == True) &
    (df_vms_az['armSkuName'].str.contains(general_purpose_machines_expression))
  ]

print(f"-> Total de registros após filtro: {len(df_vms_az)}")

-> Total de registros após filtro: 38454


In [ ]:
# Excluir colunas
columns_out = ['unitPrice', 'meterId', 'meterName', 'tierMinimumUnits', 'serviceFamily', 'region', 'effectiveStartDate', 'productId', 'skuId', 'skuName', 'serviceName', 'serviceId', 'unitOfMeasure', 'type', 'isPrimaryMeterRegion', 'name', 'osDiskSizeInMB',  'maxDataDiskCount']
df_vms_az.drop(columns=columns_out, inplace=True)

In [ ]:
# Conversão dos custos de reserva para custo / hora
df_vms_az['priceReserved1yr'] = df_vms_az['priceReserved1yr'] / 8760
df_vms_az['priceReserved3yr'] = df_vms_az['priceReserved3yr'] / 26280

df_vms_az[['priceReserved1yr', 'priceReserved3yr']] = round(df_vms_az[['priceReserved1yr', 'priceReserved3yr']], 3)

In [ ]:
# Incluindo a coluna 'extractionDate' para fins de documentação
df_vms_az['extractionDate'] = extraction_date
# Incluindo a coluna 'provider'
df_vms_az['provider'] = 'Azure'

In [ ]:
df_vms_az.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38454 entries, 2 to 109151
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype                            
---  ------                --------------  -----                            
 0   currencyCode          38454 non-null  object                           
 1   priceOnDemand         38454 non-null  float64                          
 2   armRegionName         38454 non-null  object                           
 3   location              38454 non-null  object                           
 4   productName           38454 non-null  object                           
 5   armSkuName            38454 non-null  object                           
 6   priceReserved1yr      10043 non-null  float64                          
 7   priceReserved3yr      10043 non-null  float64                          
 8   numberOfCores         38454 non-null  int64                            
 9   resourceDiskSizeInMB  38454 non-null  int64

In [ ]:
# Renomeando e reordenando colunas
df_vms_az.rename(
    columns={
        'armSkuName': 'machineType',
        'memoryInMB': 'memoryInMiB',
        'location': 'region',
        'armRegionName': 'regionCode'
    }, inplace=True
)

ordered_columns = ['extractionDate', 'provider', 'machineType', 'regionCode', 'region', 'numberOfCores', 'memoryInMiB', 'currencyCode', 'priceOnDemand', 'priceReserved1yr', 'priceReserved3yr']
df_vms_az = df_vms_az[ordered_columns]

In [ ]:
df_vms_az.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38454 entries, 2 to 109151
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype                            
---  ------            --------------  -----                            
 0   extractionDate    38454 non-null  datetime64[us, America/Sao_Paulo]
 1   provider          38454 non-null  object                           
 2   machineType       38454 non-null  object                           
 3   regionCode        38454 non-null  object                           
 4   region            38454 non-null  object                           
 5   numberOfCores     38454 non-null  int64                            
 6   memoryInMiB       38454 non-null  int64                            
 7   currencyCode      38454 non-null  object                           
 8   priceOnDemand     38454 non-null  float64                          
 9   priceReserved1yr  10043 non-null  float64                          
 10  priceReserved3

## Persistência 📁

In [ ]:
file_name = f'vms_az_{extraction_date.strftime("%d-%m-%Y")}.csv'
set_key(env_file, "AZURE_FILE", file_name)

df_vms_az.to_csv(f'/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Dados/{file_name}', index=False)